# My tests of iNaturalist API

## import observation from iNaturalist

In [2]:
import requests  # https://realpython.com/python-requests/ - lib for requests
import time
from ipyleaflet import Map, Marker, GeoJSON, basemaps # ipyleaflet in anaconda https://anaconda.org/conda-forge/ipyleaflet
from geojson import MultiPoint, Feature, FeatureCollection #geojson in anaconda - https://anaconda.org/conda-forge/geojson


# my observation
url = "https://api.inaturalist.org/v1/observations?id=72266225"

# r1 - response
r1 = requests.get(url)

# print and convert from jason to dict.
#print(r.json())

#location from tekst do list
# l - location # lon, lat 
#print(r.json()['results'][0]['location']) 
l = [float(r1.json()['results'][0]['location'].split(",")[0]),float(r1.json()['results'][0]['location'].split(",")[1])]
l

[50.04878863, 19.84250268]

## define map and marker

In [3]:
#parameters for map
center = l
zoom = 15

#map object
m = Map(center=center, zoom=zoom)

#define and add marker
marker = Marker(location=center, draggable=False)
m.add_layer(marker);

#add map
m

#to jest testowa linijka

Map(center=[50.04878863, 19.84250268], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

## add geoJSON point

In [126]:
#  !!!!!! Attention ! for unknown reasons ipyleaflet needs changed coordinates in MultiPoint !!!!!
# https://github.com/jupyter-widgets/ipyleaflet/pull/439#issuecomment-658030755
# definition of geoJSON - simple point :)
data1 = FeatureCollection( [ Feature ( geometry=MultiPoint( [(l[::-1])] ) ) ] )

#print(data1)

'''
#optional saving to file
#import json #optional saving to file
if not os.path.exists('myobserv.json'):
    with open('myobserv.json', 'w') as f:
        f.writelines(str(data1))

with open('myobserv.json', 'r') as f:
    data = json.load(f)
print(data)
'''

#ipyleaflet GeoJSON layer
geo_json = GeoJSON(data=data1, point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3})

#print(geo_json1)

m2 = Map(center=center, zoom=zoom)
m2.add_layer(geo_json)

m2

Map(center=[50.04878863, 19.84250268], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_tit…

## Get place id from iN

In [127]:
#------------------------------
# import place id from iN
place_type = "Country" #Type of place to retrieve
q = "Poland" #query value

url_q = "https://www.inaturalist.org/places.json?place_type=" + place_type + "&q=" + q
#print(url_q)

# r - response with place atributes (without geometry -> query)
r = requests.get(url_q)
#print(r.json())

place_id = r.json()[0]['id']
#print(place_id)

# import observations for spec. taxon and spec. place
#------------------------------

print("Place name:", q, ", Place ID: ", place_id)

Place name: Poland , Place ID:  7800


## Get observation from iN

In [128]:
# Anemonoides nemorosa in Poland
# get observation from iN


t = "Anemonoides nemorosa" #taxon name

url_t = "https://api.inaturalist.org/v1/observations?"
query_params = {"place_id":place_id, "taxon_name":t, "page": 1  }

#rt = requests.get(url_t, params=query_params)
#print(rt.json()["total_results"])

obs = []  #observations

#----------
# inspiracion ane while loop :https://github.com/AgentschapPlantentuinMeise/iNatBioblitzes/blob/main/inat.ipynb
#----------

page = 1
while True:
    
    #rt - object with observations from one page
    rt = requests.get(url_t, params=query_params)

    if page > num_pages: 
        print("---END---")
        break
    
    if page == 1:
        num_pages = rt.json()["total_results"] // rt.json()["per_page"] + 1

        print("Results:", rt.json()["total_results"], ", ", num_pages, " pages total")
    
    print("Retrieving page ", url_t + " page " + str(page))
    page += 1
    query_params["page"] = page
    
    for i in rt.json()["results"]:
        l = [ float(k) for k in i['location'].split(",") ]
        t = tuple(l[::-1] )
        obs.append( t )
        
    #print(t)

    time.sleep(1)


Results: 600 ,  21  pages total
Retrieving page  https://api.inaturalist.org/v1/observations? page 1
Retrieving page  https://api.inaturalist.org/v1/observations? page 2
Retrieving page  https://api.inaturalist.org/v1/observations? page 3
Retrieving page  https://api.inaturalist.org/v1/observations? page 4
Retrieving page  https://api.inaturalist.org/v1/observations? page 5
Retrieving page  https://api.inaturalist.org/v1/observations? page 6
Retrieving page  https://api.inaturalist.org/v1/observations? page 7
Retrieving page  https://api.inaturalist.org/v1/observations? page 8
Retrieving page  https://api.inaturalist.org/v1/observations? page 9
Retrieving page  https://api.inaturalist.org/v1/observations? page 10
Retrieving page  https://api.inaturalist.org/v1/observations? page 11
Retrieving page  https://api.inaturalist.org/v1/observations? page 12
Retrieving page  https://api.inaturalist.org/v1/observations? page 13
Retrieving page  https://api.inaturalist.org/v1/observations? page 

## Interactive map of iN observations

In [129]:
#Anemonoides nemorosa in Poland
#vizualization

#for i in range(10): print(obs[i])   #test firs 10 values
    
    
map_center = [ float(i) for i in [ r.json()[0]['latitude'], r.json()[0]['longitude'] ] ] 
#print(map_center)

m_poland = Map(basemap=basemaps.CartoDB.Positron, center=map_center, zoom=5)

#geoJSON object from observationns
geo_obs = FeatureCollection( [ Feature ( geometry=MultiPoint( obs ) ) ] )

#ipyleaflet GeoJSON layer
geo_json = GeoJSON(data=geo_obs, point_style={'radius': 5, 'fillOpacity': 1, 'fillColor': 'green', 'weight': 0})

m_poland.add_layer(geo_json )

m_poland


Map(center=[52.1482338661, 19.3788685716], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in…

In [130]:
#example from documentation
#https://ipyleaflet.readthedocs.io/en/latest/api_reference/geo_json.html

import os
import json
import random
import requests

from ipyleaflet import Map, GeoJSON

if not os.path.exists('europe_110.geo.json'):
    url = 'https://github.com/jupyter-widgets/ipyleaflet/raw/master/examples/europe_110.geo.json'
    r = requests.get(url)
    with open('europe_110.geo.json', 'w') as f:
        f.write(r.content.decode("utf-8"))

with open('europe_110.geo.json', 'r') as f:
    data = json.load(f)

def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }

m = Map(center=(50.6252978589571, 0.34580993652344), zoom=3)

#print(data)


geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
m.add_layer(geo_json)

m


Map(center=[50.6252978589571, 0.34580993652344], controls=(ZoomControl(options=['position', 'zoom_in_text', 'z…

In [131]:
# https://github.com/jupyter-widgets/ipyleaflet/pull/439#issuecomment-658030755

from ipyleaflet import Map, GeoJSON
m3 = Map(basemap=basemaps.OpenStreetMap.BlackAndWhite, center=(46.91, 7.43), zoom=17)

d = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "MultiPoint",
        "coordinates": [
          [
            7.43, 46.91
          ]
        ]
      }
    }
  ]
}

geo_json = GeoJSON(data=d, point_style={'radius': 5, 'fillOpacity': 0.5, 'fillColor': 'green', 'weight': 0})
m3.add_layer(geo_json)
m3

Map(center=[46.91, 7.43], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…